<a href="https://colab.research.google.com/github/kuuumaaaa/data_analyze_technique_in_kaggle/blob/main/ch02/ch02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2値分類
混同行列

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np

# 0,1で表される二値分類の真の値と予測値
y_true = [1, 0, 1, 1, 0, 1, 1, 0]
y_pred = [0, 0, 1, 1, 0, 0, 1, 1]

tp = np.sum((np.array(y_true) == 1) & (np.array(y_pred) == 1))
tn = np.sum((np.array(y_true) == 1) & (np.array(y_pred) == 0))
fp = np.sum((np.array(y_true) == 0) & (np.array(y_pred) == 1))
fn = np.sum((np.array(y_true) == 0) & (np.array(y_pred) == 0))

confusion_matrix1 = np.array([[tp,fp],[fn,tn]])
print(confusion_matrix1)

# scikit-learnの場合は混同配列の順番が異なる
confusion_matrix2 = confusion_matrix(y_true, y_pred)
print(confusion_matrix2)

[[3 1]
 [2 2]]
[[2 1]
 [2 3]]


accuracy

In [ ]:
from sklearn.metrics import accuracy_score

# 0,1で表される二値分類の真の値と予測値
y_true = [1, 0, 1, 1, 0, 1, 1, 0]
y_pred = [0, 0, 1, 1, 0, 0, 1, 1]

accuracy = accuracy_score(y_true , y_pred)
print(accuracy)

0.625


logloss

In [ ]:
from sklearn.metrics import log_loss

# 0,1で表される二値分類の真の値と予測値
y_true = [1, 0, 1, 1, 0, 1]
y_pred = [0.1, 0.2, 0.8, 0.8, 0.1, 0.3]

logloss = log_loss(y_true , y_pred)
print(logloss)

0.7135581778200728


multi accuracy

In [ ]:
from sklearn.metrics import log_loss

# 0,1で表される二値分類の真の値と予測値
y_true = [0, 2, 1, 2, 2]
y_pred = [[0.68, 0.32, 0.00],
          [0.00, 0.00, 1.00],
          [0.60, 0.40, 0.00],
          [0.00, 0.00, 1.00],
          [0.28, 0.12, 0.60],
          ]

logloss = log_loss(y_true , y_pred)
print(logloss)

0.3625557672904274


# 評価指標と目的関数の違い
- 目的関数
  - 目的関数はモデルの学習において最適化される関数であり、目的関数が最小化されるように学習が進む。
  - うまく学習を進めるためには、目的関数は微分可能である必要がある。
  - 回帰タスクはＲＭＳＥ、分類タスクはloglossが用いられることがおおい。
- 評価指標
  - モデルの良しあしを測る指標
  - ビジネス上の価値基準を用いて指標を決定することが多い
  - 予測値を変えたときのスコアの変化が数学的に扱いにくいなど、目的関数としてしようとしてもうまくいかないことが多い

  目的関数と評価指標があっている場合、評価指標に対して最適化できている。
  あっていない場合は、カスタム目的関数を用いて対応するなど必要
# カスタム目的関数
モデルやライブラリで提供されていない評価指標や目的関数を作成する必要がある。
指定することにより以下のことが実現可能
 - その関数を最小化する学習が可能
 - スコアが学習時にモニタリング可能

 APIで実装する必要あり

In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

train = pd.read_csv('/content/drive/MyDrive/参考書/Kaggleで勝つデータ分析の技術/kagglebook-master/input/ch01-titanic/train.csv')
train_x = train.drop(['Survived'],axis =1)
train_y = train['Survived']
test = pd.read_csv('/content/drive/MyDrive/参考書/Kaggleで勝つデータ分析の技術/kagglebook-master/input/ch01-titanic/test.csv')
test_x = test.copy()
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

Mounted at /content/drive


In [10]:
from  sklearn.preprocessing import LabelEncoder

# 変数PassengerIdを除去
train_x = train_x.drop(['PassengerId'],axis=1)
test_x = test_x.drop(['PassengerId'],axis=1)

# 変数Name,Ticket,Cabiを削除
train_x = train_x.drop(['Name','Ticket','Cabin'],axis=1)
test_x = test_x.drop(['Name','Ticket','Cabin'],axis=1)

# それぞれのカテゴリ変数にlabel encoding
for c in ['Sex','Embarked']:
  # 学習データに基づいてどう変換するかを定める
  le = LabelEncoder()
  le.fit(train_x[c].fillna('NA'))

  # 学習データ、テストデータを変換する
  train_x[c] = le.transform(train_x[c].fillna('NA'))
  test_x[c] = le.transform(test_x[c].fillna('NA'))

In [12]:
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
  # 学習データを学習データとバリデーションデータに分ける
  tr_x,va_x = train_x.iloc[tr_idx],train_x.iloc[va_idx]
  tr_y,va_y = train_y.iloc[tr_idx],train_y.iloc[va_idx]

In [18]:
import xgboost as xgb
from sklearn.metrics import log_loss

# 特徴量と目的変数をxgboostのデータ構造に変換
# 学習データの特徴量と目的変数がtr_x、tr_y、バリデーションデータの特徴量と目的変数がva_x、va_とする
dtrain = xgb.DMatrix(tr_x ,label=tr_y)
dvalid = xgb.DMatrix(va_x ,label=va_y)

# カスタム目的変数（この場合はloglossであり、xgboostの'binary:logistic'と等価
def logregobj(preds,dtrain):
  # 真のラベルを取得
  labels = dtrain.get_label()
  # シグモイド関数
  preds = 1.0/(1.0 + np.exp(-preds))
  # 勾配
  grad = preds - labels
  # 二階微分値
  hess = preds*(1.0 - preds)
  return grad, hess

# カスタム評価指標（この場合は誤答率）
def evalerror(preds, dtrain):
  # 真の値のラベルを取得
  labels = dtrain.get_label()
  return 'custom-error' , float(sum(labels != (preds > 0.0))) / len(labels)

# ハイパーパラメータの設定
params = {'silent': 1, 'random_state' : 71}
num_round = 50
watchlist = [(dtrain, 'train'), (dvalid,'eval')]

# モデルの学習の実行
bst = xgb.train(params, dtrain ,num_round ,watchlist ,obj = logregobj, feval=evalerror)

# 目的関数にbinary:logisticを指定した時と異なり、確率に変換する前の値で予測値が出力されるため変換が必要
pred_val = bst.predict(dvalid)
pred = 1.0 / (1.0 + np.exp(-pred_val))
logloss = log_loss(va_y, pred)
print(log_loss)

# 通常
params = {'silent' : 1, 'random_state' : 71 ,'objective' : 'binary:logistic'}
bst = xgb.train(params,dtrain,num_round,watchlist)

pred = bst.predict(dvalid)
logloss = log_loss(va_y,pred)
print(logloss)

[0]	train-rmse:0.379629	eval-rmse:0.438629	train-custom-error:0.201794	eval-custom-error:0.189189
[1]	train-rmse:0.602879	eval-rmse:0.673732	train-custom-error:0.155456	eval-custom-error:0.175676
[2]	train-rmse:0.831602	eval-rmse:0.895301	train-custom-error:0.134529	eval-custom-error:0.193694
[3]	train-rmse:1.03692	eval-rmse:1.09628	train-custom-error:0.130045	eval-custom-error:0.175676
[4]	train-rmse:1.21737	eval-rmse:1.29041	train-custom-error:0.115097	eval-custom-error:0.175676
[5]	train-rmse:1.37494	eval-rmse:1.47172	train-custom-error:0.118087	eval-custom-error:0.175676
[6]	train-rmse:1.50554	eval-rmse:1.61331	train-custom-error:0.118087	eval-custom-error:0.184685
[7]	train-rmse:1.62626	eval-rmse:1.74989	train-custom-error:0.106129	eval-custom-error:0.184685
[8]	train-rmse:1.7364	eval-rmse:1.87267	train-custom-error:0.113602	eval-custom-error:0.189189
[9]	train-rmse:1.83466	eval-rmse:1.95983	train-custom-error:0.09716	eval-custom-error:0.18018
[10]	train-rmse:1.88392	eval-rmse:1.9

# 評価指標の最適化
## 評価指標の最適化のアプローチ
- 正しくモデリング
  - 評価指標と目的関数を同じものにすることで最適化できる
- 前処理により異なる評価指標を最適化
  - 評価指標がRMSLEの場合、学習データの目的変数の対数をとって変換し、目的関数をRMSEで学習したあと、指数関数で変換
- 異なる評価指標の最適化を行ったあとに後処理
  - 学習後、評価指標の性質に基づいて計算・最適化アルゴリズムを用いてい閾値を最適化
- カスタム目的関数
- 異なる評価指標を最適化しアーリーストッピング
  - 評価がちょうど望む指標が最適になる時点で学習をやめる

## 閾値の最適化
正か負か判断する場合、閾値で判定
F1-scoreの場合、場合によって0.5が閾値ではなくなるため下記の方法などで最適な閾値を算出
- とりうる閾値すべてを調査
   - 0.01--0.99まで0.01ずつ走査
- 最適化アルゴリズム
  - scipy.optimizeなどを用いて、閾値を引数にしスコアを返す関数を最適化

目的関数が微分可能でなくても使用できる最適化アルゴリズムnelder-meadを使用する。
(cobyla:制約式を設定できるアルゴリズム
、slsqp:目的関数、製薬式が微分可能であることを必要とするアルゴリズム)

In [12]:
from sklearn.metrics import  f1_score
from scipy.optimize import minimize
# サンプルデータ生成の準備
rand = np.random.RandomState(seed = 71)
train_y_prob = np.linspace(0, 1.0, 10000)

# 真の値と予測値が以下だとする
# 条件に当てはまるものをtrue,faulseとして配列を作成
train_y =pd.Series(rand.uniform(0.0, 1.0, train_y_prob.size) < train_y_prob)
# obj.clip(配列、最小値、最大値)：配列の中の数字を最小値以上最大値以下
train_pred_prob = np.clip(train_y_prob * np.exp(rand.standard_normal(train_y_prob.shape) * 0.3),0.0 ,1.0)

init_threshold = 0.5
init_score = f1_score(train_y, train_pred_prob >= init_threshold)
print(init_threshold, init_score)

# 最適化の目的関数を設定
def f1_opt(x):
  return -f1_score(train_y, train_pred_prob >= x)

# scipy.optimizeのminimizeメソッドで最適な閾値を求める
result = minimize(f1_opt, x0=np.array([0.5]), method="Nelder-Mead")
best_threshold = result['x'].item()
best_score = f1_score(train_y, train_pred_prob >= best_threshold)
print(best_threshold, best_score)

0.5 0.7224831529507862
[0.32324219]
0.32324218749999983 0.7557317703844165
